#### FeedParserDict 객체 
- 피드 정보를 담은 feed 속성과 개별적인 항목을 담은 entries 속성을 가짐

# 크롤러 만들기
1. 웹 페이지 추출 - requests 이용
2. 웹 페이지 스크레이핑 - lxml 이용
3. 데이터 저장 - SQLite에 저장

http://www.hanbit.co.kr/store/books/new_book_list.html

In [3]:
import requests
import lxml.html

In [23]:
res = requests.get('http://www.hanbit.co.kr/store/books/new_book_list.html')
root = lxml.html.fromstring(res.content)
for a in root.cssselect('.book_tit a'):
    url_base = a.get('href')
    url = 'http://www' + url_base
    print('--')
    print(url)

--
http://www/store/books/look.php?p_code=B2098898028
--
http://www/store/books/look.php?p_code=B8295428825
--
http://www/store/books/look.php?p_code=B8928200501
--
http://www/store/books/look.php?p_code=B9061870792
--
http://www/store/books/look.php?p_code=B8030074555
--
http://www/store/books/look.php?p_code=B4906042545
--
http://www/store/books/look.php?p_code=B6282611417
--
http://www/store/books/look.php?p_code=B5034837432
--
http://www/store/books/look.php?p_code=B5940484819
--
http://www/store/books/look.php?p_code=B7654754187
--
http://www/store/books/look.php?p_code=B7950684957
--
http://www/store/books/look.php?p_code=B4364942864
--
http://www/store/books/look.php?p_code=B4833189042
--
http://www/store/books/look.php?p_code=B5719967617
--
http://www/store/books/look.php?p_code=B3780991491
--
http://www/store/books/look.php?p_code=B7592712794
--
http://www/store/books/look.php?p_code=B5203031354
--
http://www/store/books/look.php?p_code=B7019463330
--
http://www/store/books/lo

make_links_absolute()
- 상대경로를 절대경로로 바꿔준다

In [30]:
res = requests.get('http://www.hanbit.co.kr/store/books/new_book_list.html')
root = lxml.html.fromstring(res.content)
root.make_links_absolute(res.url)
for a in root.cssselect('.book_tit a'):
    url = a.get('href')
    print('--')
    print(url)

--
http://www.hanbit.co.kr/store/books/look.php?p_code=B2098898028
--
http://www.hanbit.co.kr/store/books/look.php?p_code=B8295428825
--
http://www.hanbit.co.kr/store/books/look.php?p_code=B8928200501
--
http://www.hanbit.co.kr/store/books/look.php?p_code=B9061870792
--
http://www.hanbit.co.kr/store/books/look.php?p_code=B8030074555
--
http://www.hanbit.co.kr/store/books/look.php?p_code=B4906042545
--
http://www.hanbit.co.kr/store/books/look.php?p_code=B6282611417
--
http://www.hanbit.co.kr/store/books/look.php?p_code=B5034837432
--
http://www.hanbit.co.kr/store/books/look.php?p_code=B5940484819
--
http://www.hanbit.co.kr/store/books/look.php?p_code=B7654754187
--
http://www.hanbit.co.kr/store/books/look.php?p_code=B7950684957
--
http://www.hanbit.co.kr/store/books/look.php?p_code=B4364942864
--
http://www.hanbit.co.kr/store/books/look.php?p_code=B4833189042
--
http://www.hanbit.co.kr/store/books/look.php?p_code=B5719967617
--
http://www.hanbit.co.kr/store/books/look.php?p_code=B378099

### 여러 개의 페이지를 크롤링 하기 위해 requests의 Session 객체 사용

In [71]:
import re
import time

session = requests.Session()
res = session.get('http://www.hanbit.co.kr/store/books/new_book_list.html')
root = lxml.html.fromstring(res.content)
root.make_links_absolute(res.url)
for a in root.cssselect('.book_tit a'):
    url = a.get('href')
    print('url : ', url)
    time.sleep(3)
    res = session.get(url)
    root = lxml.html.fromstring(res.content)
    title = root.cssselect('.store_product_info_box h3')[0].text
    print('제목 : ' , title)
    price = root.cssselect('.pbr strong')[0].text
    print('가격 : ', price)
    print('목차 : ')
    # 목차 출력 / 공백과 None 제외하고 출력
    for p in root.cssselect('#tabs_3 .detail_conbox p'):  # id값이 tabs_3, class갑이 detail_connox의 p태그
        content = p.text
#         print(content)
        if content != None:
            content = re.sub('\s+', '', content) # content에서 '\s+'를 ''로 바꿔라
        if content == None or content == '':
            continue
        print(p.text)
        print('-------')

url :  http://www.hanbit.co.kr/store/books/look.php?p_code=B2098898028
제목 :  나도 글 좀 잘 쓰면 소원이 없겠네
가격 :  13,320
목차 : 
프롤로그 다시, 글쓰기에 앞서
-------
인트로 당신의 어릴 적 사진 같은 글을 찍어 봅시다
-------
이 책의 핵심 구성
-------
들어가기 전에, 잠깐 점검
-------
01 내가 좋아하는 낱말 20개 이상 쓰기
-------
02 내가 좋아하는 속담이나 격언 10문장 이상 쓰기
-------
03 끝말잇기
-------
04 삼행시 쓰기
-------
05 브레인스토밍 글쓰기
-------
1주차 일기 쓰기 / 어라, 또 일기라고?
-------
1일차 보고 들은 것 쓰기
-------
2일차 감정을 문장으로 묘사하기
-------
3일차 경험한 사건을 문장으로 묘사하기
-------
4일차 보고 듣고 생각한 것 쓰기
-------
5일차 나의 하루
-------
1일차 경험을 정리하기 
-------
2일차 경험을 짧은 글로 풀어 보기
-------
3일차 기행문 쓰기
-------
4일차 경험에서 생각으로 확장하는 글쓰기
-------
5일차 나의 경험과 생각 쓰기
-------
1일차 무엇이든 보고 나면 무조건 메모하기
-------
2일차 키워드를 문장으로 정리하기
-------
3일차 좋으나 싫으나 팩트로 설명하기
-------
4일차 논리적으로 설득하기
-------
5일차 좋아하는 작품으로 리뷰 쓰기
-------
1일차 무엇을 쓸까?
-------
2일차 무슨 이야기를 어떻게 전개할까?
-------
3일차 좀 더 쉽고 재미있게 쓰자
-------
4일차 퇴고를 합시다!
-------
5일차 나만의 글쓰기
-------
01 내 자서전의 제목 쓰고 고르기
-------
02 내 주변 사람들을 한 단어로 묘사하기
-------
03 드라마나 웹툰 리뷰 쓰기
-------
04 캐릭터나 스토리를 바꾸는 글쓰기
-------
05

제목 :  IT CookBook, UI/UX 디자인 이론과 실습 with Adobe XD
가격 :  28,000
목차 : 
01. UI의 이해
-------
02. UX의 이해
-------
03. 모바일 환경의 발전
-------
04. UI/UX 디자인의 최신 트렌드
-------
01. UX 디자인 프로세스와 개발 방법
-------
02. 디자인 리서치
-------
03. 디자인 리서치 기법
-------
04. 디자인 리서치 결과 정리: 모델링
-------
05. 아이디어 도출
-------
01. UX 디자인 설계
-------
02. 정보 구조 설계
-------
03. 와이어프레임
-------
04. 프로토타이핑
-------
05. UI/UX 디자인 진행 사례
-------
01. 모바일 UI/UX 디자인의 종류
-------
02. UI/UX 디자인 프레임워크
-------
03. 모바일 운영체제의 이해
-------
04. 모바일 디자인 가이드 요소
-------
01. 앱 아키텍처
-------
02. 시각 요소
-------
03. 아이콘
-------
04. 바
-------
05. 뷰
-------
06. 컨트롤
-------
01. 내비게이션
-------
02. 아이콘
-------
03. 앱 바와 시트
-------
04. 탭과 버튼
-------
05. 리스트와 내비게이션 드로어
-------
06. 카드
-------
07. 컨트롤
-------
01. 아이콘의 이해
-------
02. iOS 아이콘 제작 1
-------
03. iOS 아이콘 제작 2
-------
04. 안드로이드 아이콘 제작 1
-------
05. 안드로이드 아이콘 제작 2
-------
01. 스플래시 스크린, 코치마크, 워크스루
-------
02. iOS 스플래시 스크린 제작 1
-------
03. iOS 스플래시 스크린 제작 2
-------
04. 안드로이드 스플래시 스크린 제작 1
-------
05. 안드로이드 스플래시 스크린 제

제목 :  IT CookBook, 스크래치로 배우는 컴퓨팅 사고
가격 :  21,000
목차 : 
01 IT 기술의 발전 
-------
02 4차 산업혁명 시대의 도래 
-------
03 4차 산업혁명의 기술 1: 빅데이터
-------
04 4차 산업혁명의 기술 2: 인공지능
-------
05 4차 산업혁명의 기술 3: 로봇
-------
06 4차 산업혁명의 기술 4: 드론
-------
07 4차 산업혁명의 기술 5: 사물인터넷
-------
08 4차 산업혁명의 기술 6: 가상현실
-------
09 4차 산업혁명의 기술 7: 증강현실
-------
10 4차 산업혁명의 기술 8: 클라우드 컴퓨팅
-------
01 컴퓨터의 개요
-------
02 컴퓨터의 발전과 변화
-------
03 컴퓨터에서의 정보의 표현
-------
01 컴퓨터를 이용한 문제 해결
-------
02 컴퓨팅 사고의 이해
-------
03 컴퓨팅 사고의 구성요소
-------
01 알고리즘의 이해
-------
02 알고리즘의 설계 및 분석
-------
03 알고리즘을 이용한 문제 해결
-------
04 프로그래밍 언어
-------
01 스크래치 소개
-------
02 스크래치 블록 사용법
-------
01 스프라이트의 구현 방식
-------
02 스프라이트의 동작 구현
-------
03 스프라이트의 형태 구현
-------
04 스프라이트의 소리 구현
-------
05 스프라이트의 펜 활용
-------
01 자료형
-------
02 연산
-------
03 변수
-------
04 지역 변수와 전역 변수
-------
05 구조적 자료형: 배열과 리스트
-------
01 조건의 이해
-------
02 문장의 이해
-------
03 문장의 반복적 실행
-------
04 문장의 선택적 실행
-------
05 반복과 선택의 중첩 실행
-------
01 프로시저
-------
02 병렬 처리
-------
01 감지
-------
02 이벤트
-

제목 :  NO BULLSHIT 선형대수학 가이드
가격 :  28,000
목차 : 
1.1 방정식 풀이
-------
1.2 수
-------
1.3 변수
-------
1.4 함수와 역함수
-------
1.5 대수학의 기본 법칙
-------
1.6 이차 방정식 풀이
-------
1.7 직교좌표 평면(직교좌표계)
-------
1.8 함수
-------
1.9 여러 가지 함수
-------
1.10 삼각함수
-------
1.11 삼각함수 항등식
-------
1.12 기하학
-------
1.13 원
-------
1.14 벡터
-------
1.15 복소수
-------
1.16 선형 연립방정식 풀이
-------
1.17 집합 표기법
-------
1장 연습문제
-------
2.1 정의
-------
2.2 벡터 연산
-------
2.3 행렬 연산
-------
2.4 선형성
-------
2.5 선형대수학 개요
-------
2장 연습문제 
-------
3.1 기약행 사다리꼴 행렬
-------
3.2 행렬방정식
-------
3.3 행렬 곱셈
-------
3.4 행렬식
-------
3.5 역행렬
-------
3장 연습문제
-------
4.1 선과 평면
-------
4.2 사영벡터
-------
4.3 사영좌표
-------
4.4 벡터공간
-------
4.5 벡터공간 기법
-------
4장 연습문제
-------
5.1 선형변환
-------
5.2 행렬로 표현하기 
-------
5.3 행렬의 기저변환
-------
5.4 가역행렬 이론 
-------
5장 연습문제
-------
6.1 고윳값과 고유벡터
-------
6.2 여러 가지 행렬 
-------
6.3 추상벡터공간 
-------
6.4 추상내적공간 
-------
6.5 그램-슈미트 직교화 
-------
6.6 행렬 분해 
-------
6.7 복소수가 포함된 선형대수학 
-------
6장 연습문제
-------
7.1 화학반응식 균형 맞추기 
-------

제목 :  저는 주식투자가 처음인데요 기본편 (2020년 개정판)
가격 :  14,850
목차 : 
개정판에 붙여
-------
머리말
-------
주식투자, 어떻게 입문할까?
-------
일광 씨의 Grow Up | 들어갑시다, 주식입문의 입문
-------
01. 주식에 대해 얼마나 알고 있나요?-돈 버는 지식을 알기 전의 지식
-------
증권과 주식이 다른 건가요? / 솔직히, 주식이 뭔가요? / 어떤 회사의 주식이든 거래할 수 있나요? / 유상증자와 무상증자가 무엇인가요?
-------
주식 플러스 지식 | 주식회사는 언제부터 시작된 것일까요?
-------
02. 주식은 어떻게 구분하나요?-주식의 종류
-------
주식의 종류는 얼마나 다양한가요? / 주주가 되면 회사 경영에 참여할 수 있나요? 
-------
03. 주식은 어디서 어떻게 거래하나요?-주식시장, 종합주가지수, 주식거래의 구조
-------
주식시장이 어디에 있는 거죠? / 종합주가지수를 왜 보라는 건가요? / 주식은 어떻게 거래되나요? / 시장관리자가 주식시장의 거래에도 관여하나요?
-------
04. 제가 직접 투자해도 괜찮을까요?-직접투자, 간접투자
-------
직접투자할까요? 간접투자할까요? / 왜 굳이 직접투자를 하나요? 
-------
주식 플러스 지식 | 간접투자의 대표적인 상품, 펀드
-------
일광 씨의 Level Up
-------
투자 원칙 새기기 | 이제 막 주식투자를 시작하려는 당신의 자세
-------
일광 씨의 Grow Up | 당신은 얼마나 위험한 사람입니까?
-------
01. 당신의 투자성향은 얼마나 위험할까요?-투자성향 파악하고 금융상품 선택하기
-------
투자성향 체크는 필수 자기진단 / 투자위험도에 따라 금융상품도 달라집니다
-------
02. 어떤 증권사가 좋을까요?-증권사 선택하고 나의 첫 주식계좌 만들기
-------
증권사에 대한 적절한 상식 / 수수료의 유혹 / 주식을 거래하는 4가지 방법 / 계좌는 어떻게 만들까요?

### time.sleep() 이용
- import time
- 크롤링 타임 지연
- () 괄호 안에 초 단위로 기입

스크레이핑 데이터 저장하기

In [76]:
import sqlite3

In [81]:
con = sqlite3.connect('C:/sqlite/testDB')
cur = con.cursor()
cur.execute('drop table if exists ebook')
cur.execute('create table ebook(url text, title text, price text, contents text)')
session = requests.Session()
res = session.get('http://www.hanbit.co.kr/store/books/new_book_list.html')

OperationalError: database is locked

In [92]:
import requests
import lxml.html
import re
import time
import sqlite3

def save_book(cur, url, title, price, contents):
    cur.execute('insert into ebook values(:url, :title, :price, :contents)',
    {'url':url, 'title':title, 'price':price, 'contents':contents})

def detail_page_scrape(url):
    res = session.get(url)
    root = lxml.html.fromstring(res.content)
    title = root.cssselect('.pbr strong')[0].text
    contents = ''
    for p in root.cssselect('#tabs_3 .detail_conbox p'):
        content = p.text
        if content != None:
            content = re.sub('\s+', '',content)
        if content == None or content == '':
            continue
        contents += p.text + '\n'
    return title,price,contents

def scrape(cur, session):
    res = session.get('http://www.hanbit.co.kr/store/books/new_book_list.html')
    root = lxml.html.fromstring(res.content)
    root.make_links_absolute(res.url)
    for a in root.cssselect('.book_tit a'):
        url = a.get('href')
        print('URL :',url)
        time.sleep(1)
        title,price,contents = detail_page_scrape(url)
        print('제목 : ', title)
        print('가격 : ', price)
        print('목차 : ')
        print(contents)
        print('----\n')
        save_book(cur, url, title, price, contents)
        
con = sqlite3.connect('C:/sqlite/testDB1')
cur = con.cursor()
cur.execute('drop table if exists ebook;')
cur.execute('create table ebook(url text, title text, price text, contents text)')
session = requests.Session()
scrape(cur, session)
con.commit()
con.close()

URL : http://www.hanbit.co.kr/store/books/look.php?p_code=B2098898028
제목 :  13,320
가격 :  14,850
목차 : 
프롤로그 다시, 글쓰기에 앞서
인트로 당신의 어릴 적 사진 같은 글을 찍어 봅시다
이 책의 핵심 구성
들어가기 전에, 잠깐 점검
01 내가 좋아하는 낱말 20개 이상 쓰기
02 내가 좋아하는 속담이나 격언 10문장 이상 쓰기
03 끝말잇기
04 삼행시 쓰기
05 브레인스토밍 글쓰기
1주차 일기 쓰기 / 어라, 또 일기라고?
1일차 보고 들은 것 쓰기
2일차 감정을 문장으로 묘사하기
3일차 경험한 사건을 문장으로 묘사하기
4일차 보고 듣고 생각한 것 쓰기
5일차 나의 하루
1일차 경험을 정리하기 
2일차 경험을 짧은 글로 풀어 보기
3일차 기행문 쓰기
4일차 경험에서 생각으로 확장하는 글쓰기
5일차 나의 경험과 생각 쓰기
1일차 무엇이든 보고 나면 무조건 메모하기
2일차 키워드를 문장으로 정리하기
3일차 좋으나 싫으나 팩트로 설명하기
4일차 논리적으로 설득하기
5일차 좋아하는 작품으로 리뷰 쓰기
1일차 무엇을 쓸까?
2일차 무슨 이야기를 어떻게 전개할까?
3일차 좀 더 쉽고 재미있게 쓰자
4일차 퇴고를 합시다!
5일차 나만의 글쓰기
01 내 자서전의 제목 쓰고 고르기
02 내 주변 사람들을 한 단어로 묘사하기
03 드라마나 웹툰 리뷰 쓰기
04 캐릭터나 스토리를 바꾸는 글쓰기
05 일상의 대화를 옮겨 적고 느낀 점 쓰기
06 기자가 되어 이슈가 된 인터넷 기사 바꿔 쓰기
07 오감이 드러나는 글쓰기
08 새로 방문한 곳에 대해 묘사하는 글쓰기
09 마음에 드는 글 필사하기
10 관점을 바꾼 일기 쓰기
11 나를 화나게 한 사건과 웃게 한 사건에 관해 기술하기
12 가장 인상적인 여행지를 골라 기행문 쓰기
13 자기 자신의 장점을 어필하는 자기 소개서 쓰기
14 내 생각과 말을 관찰하여 심리 상태 묘사하기
15 지난 4주간의 글쓰기 훈련을 마친 소감 쓰기
작가 추천 도서 목록
에필로그

제목 :  24,000
가격 :  14,850
목차 : 
01. 리눅스의 개요
02. 우분투 리눅스 배포판
03. 가상머신의 개요
04. VMware 설치
05. 가상머신 생성
06. VMware의 특징과 실습 준비
요약/연습문제
01. 우분투 데스크톱 설치
02. 우분투 서버 설치
03. 쿠분투 설치
요약/연습문제
01. 리눅스의 기초
02. 자동 완성 기능
03. vi 에디터와 도움말
04. 리눅스 마운트
요약/연습문제
01. 리눅스 기본 명령어
02. 네트워크 명령어
03. 파일 압축과 묶기
04. 시스템 설정
요약/연습문제
01. 사용자 관리
02. 파일의 소유와 허가권
03. 프로세스, 서비스(데몬)
요약/연습문제
01. 프로그램 설치 명령어 dpkg
02. 패키지 설치 명령어 apt-get
03. 응급 복구와 GRUB 부트로더
요약/연습문제
01. GNOME 데스크톱 환경 설정
02. X 윈도우 응용 프로그램의 종류
03. GNOME 데스크톱 설치
요약/연습문제
01. 우분투 소프트웨어 센터
02. KDE 데스크톱 설치와 사용
03. 리눅스에 윈도우 설치
요약/연습문제
01. 디스크와 파티션
02. 디스크 추가
03. 사용자별 공간 할당
요약/연습문제
01. RAID의 개념과 RAID 레벨
02. RAID 구축을 위한 준비
03. RAID의 다양한 레벨 구축
요약/연습문제
01. 셸의 개념과 특징
02. 셸 스크립트 프로그래밍 기본
03. 셸 스크립트 프로그래밍 응용
요약/연습문제
01. 텔넷 서버
02. SSH 서버
03. VNC 서버
요약/연습문제
01. APM의 개요와 설치
02. XE를 활용한 웹 사이트 구축
03. FTP 서버 설치와 운영
요약/연습문제
01. 미니 프로젝트의 개요
02. 실무 환경의 우분투 설치
03. RAID 6 구축
04. 백업 자동화
05. Visual Studio Code 개발 환경 구축
요약/연습문제
찾아보기

----

URL : http://www.hanbit.co.kr/store/books/look.php?p

제목 :  28,000
가격 :  14,850
목차 : 
1.1 방정식 풀이
1.2 수
1.3 수 표현
1.4 변수
1.5 함수와 역함수
1.6 대수학의 기본 법칙
1.7 2차 방정식 풀이
1.8 지수
1.9 로그
1.10 데카르트 평면
1.11 함수
1.12 함수 모음
1.13 함수 변환
1.14 기하학
1.15 삼각법
1.16 삼각 함수 공식
1.17 원
1.18 타원
1.19 쌍곡선
1.20 선형 연립 방정식 풀이
1.21 복리
1.22 집합 표기
1장 연습문제
2.1 서론
2.2 운동학
2.3 미적분학 기초
2.4 미적분학과 운동학
2장 연습문제
3.1 대자연
3.2 벡터
3.3 기저
3.4 벡터곱
3.5 복소수
3장 연습문제
4.1 서론
4.2 포물체 운동
4.3 힘
4.4 힘 다이어그램
4.5 운동량
4.6 에너지
4.7 등속 원운동
4.8 각운동
4.9 단순 조화 운동
4.10 결론
4장 연습문제
5.1 서론
5.2 개요
5.3 무한
5.4 극한
5.5 극한 공식
5.6 미분
5.7 미분 공식
5.8 미분법
5.9 고계 미분
5.10 최적화 알고리즘
5.11 음함수 미분법
5.12 적분
5.13 리만 합
5.14 미적분학의 기본 정리
5.15 적분법
5.16 적분의 응용
5.17 이상 적분
5.18 수열
5.19 급수
5.20 결론
5장 연습문제
부록 A 표기
부록 B 상수, 단위, 변환 비율
부록 C SymPy 지도서
부록 D 공식

----

URL : http://www.hanbit.co.kr/store/books/look.php?p_code=B7019463330
제목 :  13,500
가격 :  14,850
목차 : 
먼저 크리에이터를 알아보자! 
그럼 키즈 크리에이터는 뭘까? 
평범했던 라임이네 가족
유튜브가 눈에 보이다! 
끼가 많던 라임이 
라임파파가 운영하던 장난감 채널
라임이가 등장하자 조회수가 2배 증가했어요! 
유튜브 키즈 데이 행사를 통해 한 단계 성장하다! 
라임이가 나온 영상의 첫 조회수는 10회 
라임이의 눈높이에 맞춘 

### 실습
-https://comic.naver.com/index.nhn
- 제목, 작가, 내용, 장르, 연령 정보 추출

In [107]:
import requests
import lxml.html
import re
import time
import sqlite3

def save_webtoon(cur, title, writer, topic, genre, age):
    cur.execute('insert into webtoon values(:title, :writer, :topic, :genre, :age)',
    {'title':title, 'writer':writer, 'topic':topic, 'genre':genre, 'age':age})

def detail_page_scrape(url):
    res = session.get(url)
    root = lxml.html.fromstring(res.content)
    title = root.cssselect('.detail h2')[0].text
    title = title.strip()
    writer = root.cssselect('.wrt_nm')[0].text
    genre = root.cssselect('.genre')[0].text
    age = root.cssselect('.age')[0].text
    contents = ''
    for p in root.cssselect('.detail p'):
        content = p.text
        if content != None:
            content = content.strip()
        if content == None or content == '':
            continue
        contents += p.text + '\n'
    return title, contents, writer , genre, age

def scrape(cur, session):
    res = session.get('https://comic.naver.com/webtoon/weekday.nhn')
    root = lxml.html.fromstring(res.content)
    root.make_links_absolute(res.url)
    for a in root.cssselect('.thumb a'):
        url = a.get('href')
        print(url)
        title, contents, writer, genre, age = detail_page_scrape(url)
        print('제목 : ', title)
        print('작가 : ', writer)
#         print('내용 : ', topic)
        print('장르 : ', genre)
        print('연령대 : ',age)
        print('내용 :',contents)
        print('----\n')
        save_webtoon(cur, title, writer, contents, genre, age)
        
con = sqlite3.connect('C:/sqlite/testDB1')
cur = con.cursor()
cur.execute('drop table if exists webtoon;')
cur.execute('create table webtoon(title text, writer text, topic text, genre text, age text)')
session = requests.Session()
scrape(cur, session)
con.commit()
con.close()

https://comic.naver.com/webtoon/list.nhn?titleId=183559&weekday=mon
제목 :  신의 탑
작가 :  
							SIU
장르 :  스토리, 판타지
연령대 :  12세 이용가
내용 : 자신의 모든 것이었던 소녀를 쫓아 탑에 들어온 소년

----

https://comic.naver.com/webtoon/list.nhn?titleId=602910&weekday=mon
제목 :  윈드브레이커
작가 :  
							조용석
장르 :  스토리, 드라마, 스포츠
연령대 :  12세 이용가
내용 : 자유를 꿈꾸는 청춘들의 스트릿라이딩 드라마

----

https://comic.naver.com/webtoon/list.nhn?titleId=648419&weekday=mon
제목 :  뷰티풀 군바리
작가 :  
							설이 / 윤성원
장르 :  스토리, 드라마
연령대 :  15세 이용가
내용 : '여자도 군대에 간다면?'본격 여자도 군대 가는 만화!

----

https://comic.naver.com/webtoon/list.nhn?titleId=733766&weekday=mon
제목 :  인생존망
작가 :  
							박태준 / 전선욱
장르 :  에피소드, 판타지, 액션
연령대 :  15세 이용가
내용 : 학교 다닐때 그렇게 놀았으면, 

----

https://comic.naver.com/webtoon/list.nhn?titleId=597478&weekday=mon
제목 :  평범한 8반
작가 :  
							영파카
장르 :  스토리, 개그, 드라마
연령대 :  15세 이용가
내용 : 안 평범한 아이들이 모인 평범한 8반

----

https://comic.naver.com/webtoon/list.nhn?titleId=694946&weekday=mon
제목 :  귀전구담
작가 :  
							QTT
장르 :  옴니버스, 판타지, 드라마, 감성, 스릴러
연령대 :  12세 이용가
내용 : 사람들

제목 :  랜덤채팅의 그녀!
작가 :  
							박은혁
장르 :  스토리, 일상, 개그, 드라마
연령대 :  15세 이용가
내용 : 랜덤채팅만 하던 준우에게 온 기적!

----

https://comic.naver.com/webtoon/list.nhn?titleId=642653&weekday=tue
제목 :  사이드킥 2~3
작가 :  
							신의철
장르 :  스토리, 판타지, 액션
연령대 :  15세 이용가
내용 : 슈퍼히어로의 조수 '사이드킥' 

----

https://comic.naver.com/webtoon/list.nhn?titleId=683496&weekday=tue
제목 :  신도림
작가 :  
							오세형
장르 :  스토리, 판타지, 액션
연령대 :  15세 이용가
내용 : 세상이 무너지고 세워진 지하도시 신도림

----

https://comic.naver.com/webtoon/list.nhn?titleId=698918&weekday=tue
제목 :  원주민 공포만화
작가 :  
							원주민
장르 :  스토리, 개그, 스릴러
연령대 :  15세 이용가
내용 : 끝까지 방심하지 말 것!

----

https://comic.naver.com/webtoon/list.nhn?titleId=730656&weekday=tue
제목 :  사신소년
작가 :  
							류
장르 :  스토리, 판타지, 액션
연령대 :  15세 이용가
내용 : 수명을 대가로 죽은 자의 능력을 빌리는 소년

----

https://comic.naver.com/webtoon/list.nhn?titleId=602287&weekday=tue
제목 :  신의 언어
작가 :  
							장래혁
장르 :  스토리, 판타지
연령대 :  12세 이용가
내용 : 퇴마계의 신개념 영웅!

----

https://comic.naver.com/webtoon/list.nhn?titleId=695321&weekday=tue
제목 :  빙탕후루
작가 :  
	

제목 :  헬퍼 2 : 킬베로스
작가 :  
							삭
장르 :  스토리, 액션
연령대 :  18세 이용가
내용 : 역시는 역시 역시군.

----

https://comic.naver.com/webtoon/list.nhn?titleId=710747&weekday=wed
제목 :  세상은 돈과 권력
작가 :  
							한동우 / 이도희
장르 :  에피소드, 일상, 드라마, 스릴러
연령대 :  15세 이용가
내용 : 재벌 3세가 일반 고등학교에 들어오면서, 선생들과 학생들은 거대 자본에 굴복해 권력을 차지하기 위한 '두뇌싸움'이 난무하는 학교로 바뀐다. 이 학교의 법칙을 부수기 위해 천재 '단건우'가 전학온다.

----

https://comic.naver.com/webtoon/list.nhn?titleId=701535&weekday=wed
제목 :  격기3반
작가 :  
							이학
장르 :  스토리, 스포츠
연령대 :  12세 이용가
내용 : 한국이 세계 격투기의 중심이 된 세상. 전국 탑 클래스 격투기 학생들만 모인 남일고에 약골 소년 주지태가 돌연 투입된다.

----

https://comic.naver.com/webtoon/list.nhn?titleId=723714&weekday=wed
제목 :  용비불패 완전판
작가 :  
							류기운 / 문정후
장르 :  스토리, 판타지, 액션, 시대극
연령대 :  15세 이용가
내용 : 전설적 명작의 완벽한 부활! 를 기다리며 정주행 START!

----

https://comic.naver.com/webtoon/list.nhn?titleId=603159&weekday=wed
제목 :  레사 시즌2~3
작가 :  
							POGO
장르 :  스토리, 판타지
연령대 :  12세 이용가
내용 : 종말의 때가 왔다...!

----

https://comic.naver.com/webtoon/list.nhn?titleId=728126&weekday=wed
제목 :  고삼무쌍
작가 :  


제목 :  금요일 베스트
작가 :  
							배진수
장르 :  옴니버스, 스릴러
연령대 :  15세 이용가
내용 : 당신의 상상을 조각내는 세상에 없던 반전 공포의 진수 '금요일'

----

https://comic.naver.com/webtoon/list.nhn?titleId=131385&weekday=thu
제목 :  쿠베라
작가 :  
							카레곰
장르 :  스토리, 판타지
연령대 :  전체연령가
내용 : 신의 이름을 가진 소녀와 마법사들이 펼쳐나가는

----

https://comic.naver.com/webtoon/list.nhn?titleId=670144&weekday=thu
제목 :  가담항설
작가 :  
							랑또
장르 :  스토리, 개그, 판타지, 시대극
연령대 :  12세 이용가
내용 : 이번 주인공은 돌이다!

----

https://comic.naver.com/webtoon/list.nhn?titleId=727798&weekday=thu
제목 :  겟백
작가 :  
							세윤
장르 :  에피소드, 액션, 드라마, 스릴러
연령대 :  15세 이용가
내용 : 7년 전, 운명이 뒤바뀐 소녀들. 

----

https://comic.naver.com/webtoon/list.nhn?titleId=701700&weekday=thu
제목 :  트롤트랩
작가 :  
							유비
장르 :  스토리, 판타지, 액션
연령대 :  12세 이용가
내용 : 트롤을 잡아, 인류를 지켜라!

----

https://comic.naver.com/webtoon/list.nhn?titleId=728619&weekday=thu
제목 :  병의 기록
작가 :  
							베어리
장르 :  에피소드, 일상, 드라마, 감성
연령대 :  12세 이용가
내용 : 충성! 이건 진짜 

----

https://comic.naver.com/webtoon/list.nhn?titleId=730389&weekday=thu
제목 :  어쩔꼰대
작가 

제목 :  화장 지워주는 남자
작가 :  
							이연
장르 :  에피소드
연령대 :  전체연령가
내용 : 밋밋한 얼굴의 대학생이 천재 메이크업 아티스트의 뮤즈?!

----

https://comic.naver.com/webtoon/list.nhn?titleId=700844&weekday=fri
제목 :  걸어서 30분
작가 :  
							이온도
장르 :  스토리, 드라마, 로맨스
연령대 :  전체연령가
내용 : 은하학원 위성은과 지구봉의 귀갓길 로맨스. 

----

https://comic.naver.com/webtoon/list.nhn?titleId=734040&weekday=fri
제목 :  현혹
작가 :  
							홍작가
장르 :  스토리
연령대 :  15세 이용가
내용 : 매혹적인 여인의 초상화 의뢰를 맡은 화가. 

----

https://comic.naver.com/webtoon/list.nhn?titleId=711422&weekday=fri
제목 :  삼국지톡
작가 :  
							무적핑크 / 이리
장르 :  에피소드, 개그, 시대극
연령대 :  12세 이용가
내용 : "유비, 조조, 손권 등 영웅들이 활약하던 후한, 그 난세에 핸드폰이 있었다면?

----

https://comic.naver.com/webtoon/list.nhn?titleId=723790&weekday=fri
제목 :  금붕어
작가 :  
							명랑 / 애풍
장르 :  스토리, 액션, 드라마
연령대 :  15세 이용가
내용 : 시한부 판정을 받게 된 킬러. 조득만.

----

https://comic.naver.com/webtoon/list.nhn?titleId=729044&weekday=fri
제목 :  난약
작가 :  
							정현주 / 박진환
장르 :  스토리, 판타지, 액션
연령대 :  15세 이용가
내용 : 진짜를 보는 눈, 

----

https://comic.naver.com/webtoon/list.nhn?titleId=5

제목 :  어글리후드
작가 :  
							미애
장르 :  스토리, 개그, 판타지, 액션
연령대 :  12세 이용가
내용 : 외계인이 유일신으로 군림한 미친 세상, 

----

https://comic.naver.com/webtoon/list.nhn?titleId=616239&weekday=sat
제목 :  윌유메리미
작가 :  
							마인드C
장르 :  에피소드, 일상, 개그
연령대 :  전체연령가
내용 : 외모는 상남자, 마음은 감성소녀 윌.외모는 청순녀, 마음은 터프가이 메리! 

----

https://comic.naver.com/webtoon/list.nhn?titleId=700843&weekday=sat
제목 :  우리 오빠는 아이돌
작가 :  
							성은
장르 :  스토리, 개그, 드라마, 로맨스
연령대 :  전체연령가
내용 : 덕계못이라고?! 집념의 덕후 남다름, 오빠의 회사에 취직하다!

----

https://comic.naver.com/webtoon/list.nhn?titleId=715159&weekday=sat
제목 :  갓핑크
작가 :  
							이상신 / 국중록
장르 :  스토리, 개그, 판타지, 액션
연령대 :  15세 이용가
내용 : 흥분하면 거대해지는 그놈!

----

https://comic.naver.com/webtoon/list.nhn?titleId=733280&weekday=sat
제목 :  나를 바꿔줘
작가 :  
							이지호 / 호띠
장르 :  스토리, 판타지, 드라마
연령대 :  15세 이용가
내용 : 인생을 바꾸기 위해 얼굴을 훔치다

----

https://comic.naver.com/webtoon/list.nhn?titleId=724274&weekday=sat
제목 :  먹이
작가 :  
							외눈박이 / 박수봉
장르 :  옴니버스, 판타지, 스릴러
연령대 :  15세 이용가
내용 : ‘불길한 것들이 현실로 넘어오기 시작했다’ 

----

https://comic.

제목 :  인어를 위한 수영교실
작가 :  
							용찬
장르 :  스토리, 판타지, 드라마, 로맨스
연령대 :  전체연령가
내용 : 헤엄에 서툰 인어를 위한 수영 교실! 

----

https://comic.naver.com/webtoon/list.nhn?titleId=733728&weekday=sun
제목 :  강남도깨비
작가 :  
							이경민 / 송준혁
장르 :  스토리, 판타지, 액션
연령대 :  15세 이용가
내용 : 전세계 44명, 황금의 피를 가진 전교1등을 노리는 도깨비가 나타났다.

----

https://comic.naver.com/webtoon/list.nhn?titleId=737021&weekday=sun
제목 :  마도
작가 :  
							이밤
장르 :  스토리, 판타지, 스릴러
연령대 :  15세 이용가
내용 : "그 얼굴, 안 쓸 거면 나줘!"

----

https://comic.naver.com/webtoon/list.nhn?titleId=732955&weekday=sun
제목 :  닥터앤닥터 육아일기
작가 :  
							닥터베르
장르 :  에피소드, 일상, 개그, 드라마
연령대 :  전체연령가
내용 : 산부인과 의사 엄마의 임신과 출산, 공학박사 아빠의 논문 기반 육아.

----

https://comic.naver.com/webtoon/list.nhn?titleId=626906&weekday=sun
제목 :  패밀리 사이즈
작가 :  
							남지은 / 김인호
장르 :  에피소드, 일상
연령대 :  전체연령가
내용 : 삼형제와 이제막 태어난 막내딸까지!

----

https://comic.naver.com/webtoon/list.nhn?titleId=703844&weekday=sun
제목 :  가비지타임
작가 :  
							2사장
장르 :  스토리, 드라마, 스포츠
연령대 :  전체연령가
내용 : 열혈따윈 개나 줘.

----

https://comic.naver.com/webtoon

In [113]:
import pdfminer

In [ ]:
https://buildmedia.readthedocs.org/

In [115]:
from pdfminer.layout import LAParams
from pdfminer.converter import PDFPageAggregator
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfpage import PDFPage
from pdfminer.layout import LTTextBox

In [ ]:
document = open('C:/data/pdfminer-docs.pdf', 'rb')
rsrcmgr = PDFResourceManager()
laparams = LAParams()
device = PDF